In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import *
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import astropy
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import constants as const
import hdbscan
from astropy.io import fits
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [6]:
GrID = pd.read_csv('rcsed_iGrID.csv')
NANfilter = pd.notna(GrID.iGrID)
y = GrID[NANfilter]
DATA = pd.read_csv('real_data.csv', index_col='Unnamed: 0')
x = DATA[NANfilter]
x = x.drop(['ind', 'spid_lamost', 'z_err', 'specObjID_sdss', 'z_sdss', 'f_spec_lega_c', 'f_z_lega_c', 'phot_data_new_galex', 'bestObjID_sdss', 'specid_6df', 'q_z_2df', 'recno_uzc', 'OBJNO_deep2', 'OBJNO_deep3', 'obsid_lamost', 'lmjd_lamost', 'mjd_lamost', 'quality_6df', 'Q_wigglez', 'NQ_gama', 'fibermag_i_sdss', 'fibermag_z_sdss', 'fibermagerr_i_sdss', 'fibermagerr_z_sdss', 'fiber2mag_i_sdss', 'fiber2mag_z_sdss', 'fiber2magerr_i_sdss', 'fiber2magerr_z_sdss', 'petromag_i_sdss', 'petromag_z_sdss', 'petromagerr_i_sdss', 'petromagerr_z_sdss', 'petror50_u_sdss', 'petror50_g_sdss', 'petror50_r_sdss', 'petror50_i_sdss', 'petror50_z_sdss' , 'multiple_z', 'dz', 'wigglez_photo'], axis=1)
x = x.drop(x.columns[x.isna().any().values].values, axis=1)
x = x.select_dtypes(include='number')

C:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
x = x.drop(["plate_sdss", "mjd_sdss", "fiberID_sdss", "fiberid_lamost"], axis=1)

In [11]:
x = x.reset_index(drop=True)
y = y.reset_index(drop=True)

AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

In [8]:
from sklearn.preprocessing import StandardScaler
nrmlzr = StandardScaler()
x = nrmlzr.fit_transform(x)
display(x)

array([[-2.76854312, -1.22562969, -0.20610424, ...,  0.09783831,
         0.0143573 ,  0.36430353],
       [-2.76854266, -1.23700874, -0.26048026, ..., -0.0622559 ,
        -0.02968761,  0.76943537],
       [-2.76853047, -0.17891901,  2.20195971, ...,  0.33816764,
         0.21332888,  0.11459488],
       ...,
       [ 2.61864792, -0.43299921,  0.66577757, ..., -0.01650083,
        -0.00793866,  0.79173905],
       [ 2.61866992, -1.25144744, -0.01599225, ...,  0.09131227,
         0.01297885,  1.01963995],
       [ 2.61868219, -1.86083405, -0.03283774, ..., -0.04332068,
        -0.04487614,  1.00003347]])

In [9]:
display(y)

,iGrID
59,50126.0
62,129111.0
72,212738.0
92,51045.0
99,203801.0
101,50126.0
103,214159.0
108,50126.0
116,22930.0
123,50752.0


In [10]:
x.shape

(401148, 62)

In [12]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)

        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 62))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [13]:
vae = VAE(x_dim=62, h_dim1= 45, h_dim2=30, z_dim=7)
if torch.cuda.is_available():
    vae.cuda()

optimizer = optim.Adam(vae.parameters())
train_loader = torch.utils.data.DataLoader(dataset=x, batch_size=1000, shuffle=True)

def loss_function(recon_x, x, mu, log_var):
    criterion = nn.MSELoss().cuda()
    BCE = criterion(recon_x, x.view(-1, 62))
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx in range(400):
        #data = torch.tensor(x.sample(n=1000).astype(np.float32).values)
        data = torch.tensor(x[np.random.choice(range(x.shape[0]), 1000)].astype(np.float32))
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [14]:
for epoch in range(1, 5):
    train(epoch)

Train Epoch: 1 [0/401148 (0%)]	Loss: 0.092240
Train Epoch: 1 [100000/401148 (25%)]	Loss: 0.002868
Train Epoch: 1 [200000/401148 (50%)]	Loss: 0.002187
Train Epoch: 1 [300000/401148 (75%)]	Loss: 0.000998
====> Epoch: 1 Average loss: 0.5989
Train Epoch: 2 [0/401148 (0%)]	Loss: 0.001626
Train Epoch: 2 [100000/401148 (25%)]	Loss: 0.001078
Train Epoch: 2 [200000/401148 (50%)]	Loss: 0.000785
Train Epoch: 2 [300000/401148 (75%)]	Loss: 0.001171
====> Epoch: 2 Average loss: 0.0015
Train Epoch: 3 [0/401148 (0%)]	Loss: 0.002339
Train Epoch: 3 [100000/401148 (25%)]	Loss: 0.000746
Train Epoch: 3 [200000/401148 (50%)]	Loss: 0.000677
Train Epoch: 3 [300000/401148 (75%)]	Loss: 0.000733
====> Epoch: 3 Average loss: 0.0011
Train Epoch: 4 [0/401148 (0%)]	Loss: 0.000911
Train Epoch: 4 [100000/401148 (25%)]	Loss: 0.000656
Train Epoch: 4 [200000/401148 (50%)]	Loss: 0.001092
Train Epoch: 4 [300000/401148 (75%)]	Loss: 0.000773
====> Epoch: 4 Average loss: 0.0011


In [15]:
mu, log_var = vae.encoder(torch.tensor(x.astype(np.float32)))
embed = vae.sampling(mu, log_var).detach().numpy()

In [16]:
display(embed)

array([[ 0.16447295, -0.88133496,  1.0313933 , ..., -0.33865055,
        -0.17480463, -0.5728178 ],
       [ 1.0925106 ,  0.19881423,  0.02260388, ...,  0.02729067,
         1.4517192 ,  0.31191292],
       [ 1.8581895 , -1.5041358 ,  0.8280001 , ...,  0.4780128 ,
        -0.89910483, -0.3174661 ],
       ...,
       [-2.2284477 , -0.5752071 ,  0.2339124 , ..., -0.15768902,
         2.0764897 , -0.6929708 ],
       [ 1.3544052 ,  0.18003255, -0.61603814, ..., -0.5695588 ,
         1.920867  ,  0.7287321 ],
       [-1.7260247 , -0.703662  ,  0.45988968, ...,  0.23031314,
        -1.9716207 , -0.5583463 ]], dtype=float32)

In [17]:
hdbScan = hdbscan.HDBSCAN(allow_single_cluster=True, min_samples=1).fit(embed)

In [18]:
homogeneity_completeness_v_measure(y.iGrID, hdbScan.labels_)

(0.0024752949749627675, 0.7463225981915803, 0.00493422482585403)

In [28]:
from tqdm.notebook import tqdm
gridRes = []
for alpha in tqdm(np.linspace(0.1, 3, 7)):
        hdbScan = hdbscan.HDBSCAN(min_samples=1, alpha=alpha, allow_single_cluster=True).fit(embed)
        gridRes.append((homogeneity_completeness_v_measure(y.iGrID, hdbScan.labels_), calinski_harabasz_score(embed, hdbScan.labels_), [alpha]))

In [29]:
print(gridRes)

[((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.1]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.5833333333333334]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [1.0666666666666667]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [1.55]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [2.033333333333333]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [2.5166666666666666]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [3.0])]


In [33]:
from tqdm.notebook import tqdm
gridRes = []
for alpha in tqdm(np.linspace(1e-7, 1e-2, 6)):
        hdbScan = hdbscan.HDBSCAN(min_samples=1, alpha=alpha, allow_single_cluster=True).fit(embed)
        gridRes.append((homogeneity_completeness_v_measure(y.iGrID, hdbScan.labels_), calinski_harabasz_score(embed, hdbScan.labels_), [alpha]))

In [34]:
print(gridRes)

[((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [1e-07]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.00200008]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.004000060000000001]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.006000040000000001]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.00800002]), ((0.0024752949749627675, 0.7463225981915803, 0.00493422482585403), 1.8672345153667382, [0.01])]


In [38]:
vae = VAE(x_dim=62, h_dim1= 45, h_dim2=30, z_dim=7)
if torch.cuda.is_available():
    vae.cuda()

optimizer = optim.Adam(vae.parameters())
train_loader = torch.utils.data.DataLoader(dataset=x, batch_size=1000, shuffle=True)

for epoch in range(1, 30):
    train(epoch)

Train Epoch: 1 [0/401148 (0%)]	Loss: 0.107453
Train Epoch: 1 [100000/401148 (25%)]	Loss: 0.003787
Train Epoch: 1 [200000/401148 (50%)]	Loss: 0.001813
Train Epoch: 1 [300000/401148 (75%)]	Loss: 0.000800
====> Epoch: 1 Average loss: 0.0069
Train Epoch: 2 [0/401148 (0%)]	Loss: 0.000782
Train Epoch: 2 [100000/401148 (25%)]	Loss: 0.000763
Train Epoch: 2 [200000/401148 (50%)]	Loss: 0.000652
Train Epoch: 2 [300000/401148 (75%)]	Loss: 0.000713
====> Epoch: 2 Average loss: 0.0013
Train Epoch: 3 [0/401148 (0%)]	Loss: 0.000866
Train Epoch: 3 [100000/401148 (25%)]	Loss: 0.001793
Train Epoch: 3 [200000/401148 (50%)]	Loss: 0.000770
Train Epoch: 3 [300000/401148 (75%)]	Loss: 0.001055
====> Epoch: 3 Average loss: 0.0013
Train Epoch: 4 [0/401148 (0%)]	Loss: 0.000745
Train Epoch: 4 [100000/401148 (25%)]	Loss: 0.002262
Train Epoch: 4 [200000/401148 (50%)]	Loss: 0.000606
Train Epoch: 4 [300000/401148 (75%)]	Loss: 0.000627
====> Epoch: 4 Average loss: 0.0014
Train Epoch: 5 [0/401148 (0%)]	Loss: 0.002092
Tr

KeyboardInterrupt: 